Introducción
¡Felicidades! Has llegado a la última sesión de este módulo. En esta última sesión vamos a trabajar con otra fuente de adquisición de datos con la que probablemente te toparás muy seguido: las bases de datos. Vamos a aprender a leer datos de una base de datos MySQL y a convertirlos a DataFrames. Recuerda que la maravilla de los DataFrames es que una vez que tus datos se encuentran en ese formato todas las técnicas que hemos aprendido durante este curso se pueden aplicar de manera idéntica.

También vamos a aprovechar que el dataset de esta sesión está dividido en múltiples tablas para aprender a unir DataFrames usando el método merge.

Vamos a finalizar con una de las herramientas más poderosas que nos ofrece pandas: las agrupaciones. Vamos a aprender a agrupar y a agregar nuestros grupos para obtener nuevos DataFrames e información útil.

Objetivos
Leer una base de datos MySQL usando un programa de Python.
Convertir datos provenientes de una base de datos en DataFrames.
Unir múltiples tablas usando el método merge.
Usar grouby para agrupar nuestros datos.
Aplicar agregaciones a nuestros grupos.
MySQL
Como seguramente ya sabrás, MySQL es un Sistema de Gestión de Bases de Datos. Según Wikipedia, una base de datos es "un conjunto de datos pertenecientes a un mismo contexto y almacenados sistemáticamente para su posterior uso". Cuando estés trabajando en startups o compañías más grandes, trabajando en proyectos a largo plazo o colaborando con científicos en sus investigaciones, muy probablemente te toparás con la necesidad de almacenar grandes cantidades de datos de una manera que permita el fácil acceso. Usar un SGBD, como MySQL, PostgreSQL, MariaDB, MongoDB, etc, va a ser la mejor opción. Esto se debe a que un SGBD puede correr no solamente en tu computadora local, sino en un servidor remoto. Al tener tu base de datos disponible en un servidor, tanto tú como los demás miembros de tu equipo pueden tener acceso a los datos rápidamente.

El día de hoy vamos a estar trabajando con una base de datos que está almacenada en un servidor remoto. Todos los procesos de adquisición de datos se realizarán a través de una librería que permite la conexión entre mi programa y el SGBD.

MySQL Connector
Para conectarnos a una base de datos, lo primero que necesitamos es, obviamente, un Sistema de Gestión de Bases de Datos que esté corriendo en algún servidor y que contenga los datos que queremos obtener. Para este Prework me voy a conectar a una instancia de MySQL que está corriendo en mi computadora en un servidor local. Durante el Work trabajarás con una base de datos que ha sido instanciada en un servidor remoto.

La base de datos con la que vamos a trabajar proviene del siguiente link: Movielens dataset. Vamos a utilizar el conjunto de datos llamado ml-1m. Este dataset contiene 3 tablas: "movies", "users" y "ratings".

La tabla "movies" contiene datos acerca de películas y sus géneros. La tabla "users" contiene datos acerca de algunos usuarios que realizaron valoraciones (ratings) de dichas películas. Finalmente, la tabla "ratings" contiene los datos de dichas valoraciones.

Otras dos tablas fueron creadas en la base de datos para tener cierta información de decodificación de algunas de las columnas de la tabla "users". Estas tablas llevan como nombre "age_ranges" y "occupations".

Vamos a utilizar la librería MySQL Connector para realizar la conexión a la base de datos. Instala la librería corriendo el siguiente comando:

!pip install mysql-connector-python

In [1]:
import pandas as pd

Ahora, en nuestro programa de Python, tenemos que importar la librería:

In [2]:
import mysql.connector

Ahora, para conectarnos a la base de datos, necesitas cierta información acerca del servidor que está corriendo la base de datos y el usuario que tiene acceso a ella. Más específicamente, mysql.connector te pide el host y port (el dominio y puerto donde está corriendo tu servidor), el user (el usuario o uno de los usuarios que tiene permiso de acceso a la base de datos), la password (la contraseña de dicho usuario) y la database (la base de datos almacenada en MySQL a la que queremos acceder). En el caso de mi servidor local, ésta es la información adecuada:

In [3]:
cnx = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password='123456',
    database='movielens'
)

Ya que la conexión está realizada, tengo que crear un objeto cursor, que es el encargado de realizar las operaciones de consulta y modificación de la base de datos:

In [4]:
cursor = cnx.cursor()

Ahora podemos simplemente utilizar este objeto cursor para correr código SQL y realizar operaciones con nuestra base de datos. Lenguaje SQL no es el tema de este módulo, pero basta con conocer un solo comando para poder solicitar la información contenida en una de las tablas de nuestra base de datos:

In [5]:
cursor.execute("SHOW TABLES")

In [6]:
cursor.fetchall()

[('age_ranges',), ('movies',), ('occupations',), ('ratings',), ('users',)]

In [7]:
cursor.execute("SELECT * FROM movies")

El comando SELECT * FROM nombre_de_la_tabla sirve para pedir todas (*) las entradas en la tabla nombre_de_la_tabla. Usualmente el lenguaje SQL se utiliza también para filtrar y estructurar nuestros datos, pero nosotros vamos a realizar esos procesos con pandas. Una vez que el comando SQL ha sido ejecutado, podemos usar el método fetchall para obtener nuestros datos:

In [8]:
result = cursor.fetchall()

Después, hay que cerrar nuestro cursor ya que hayamos terminado:

In [9]:
cursor.close()

True

Ahora, ¿qué es lo que hay almacenado en nuestra variable result? Vamos a ver el tipo de dato:

In [10]:
type(result)

list

¡Es una lista!

Veamos el primer elemento de nuestra lista:

In [11]:
result[0]

(1, 'Toy Story (1995)', "Animation|Children's|Comedy")

Es una tupla. Una tupla es un tipo de estructura de datos de Python que es básicamente una lista que no puede ser modificada. Afortunadamente, pandas puede recibir listas de tuplas como input para crear DataFrames. Lo único que necesita es que le digamos el nombre de cada columna al momento de crear el DataFrame. Dicha información acerca de las tablas viene incluida en el Readme del dataset . Vamos entonces a crear nuestro DataFrame usando los datos obtenidos:

In [12]:
movies = pd.DataFrame(result, columns=['movie_id', 'title', 'genres'])

In [13]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


¡Éxito!

Vamos entonces a usar el mismo proceso para adquirir todas las tablas que hay en nuestro dataset y convertirlas a DataFrames:

In [14]:
cursor = cnx.cursor()

# users
cursor.execute("SELECT * FROM users")
result = cursor.fetchall()
users = pd.DataFrame(result, columns=['user_id', 'gender', 'age', 'occupation', 'cp'])

# ratings
cursor.execute("SELECT * FROM ratings")
result = cursor.fetchall()
ratings = pd.DataFrame(result, columns=['user_id', 'movie_id', 'rating', 'timestamp'])

# age ranges
cursor.execute("SELECT * FROM age_ranges")
result = cursor.fetchall()
age_ranges = pd.DataFrame(result, columns=['age_id', 'range'])

# occupations
cursor.execute("SELECT * FROM occupations")
result = cursor.fetchall()
occupations = pd.DataFrame(result, columns=['occupation_id', 'description'])

cursor.close()

True

In [15]:
users

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [17]:
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [18]:
age_ranges

,age_id,range
0,1,Under 18
1,18,18-24
2,25,25-34
3,35,35-44
4,45,45-49
5,50,50-55
6,56,56+


In [16]:
occupations

,occupation_id,description
0,0,other or not specified
1,1,academic/educator
2,2,artist
3,3,clerical/admin
4,4,college/grad student
5,5,customer service
6,6,doctor/health care
7,7,executive/managerial
8,8,farmer
9,9,homemaker


merge

Ya aprendimos a concatenar DataFrames usando el método concat. Ahora vamos a conocer merge que nos permite realizar uniones entre DataFrames de una manera muy similar a como funcionan los joins en SQL.

Primero vamos a ver qué hay dentro de algunas de nuestras tablas:

In [19]:
users

,user_id,gender,age,occupation,cp
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


Tenemos 6040 entradas de usuarios con diversos datos acerca de ellos. Pero tenemos, por ejemplo, una columna "occupation" que sólo contienen ids, no los nombres reales de las ocupaciones. Por suerte tenemos otra tabla llamada occupations que contiene esa información:

In [20]:
occupations

,occupation_id,description
0,0,other or not specified
1,1,academic/educator
2,2,artist
3,3,clerical/admin
4,4,college/grad student
5,5,customer service
6,6,doctor/health care
7,7,executive/managerial
8,8,farmer
9,9,homemaker


Primero que nada, me he percatado de que user_id y occupation_id podrían ser los índices de sus respectivos DataFrames. Voy a modificarlos para que así sea y evitar tener información redundante:

In [ ]:
users = users.set_index('user_id', drop=True)

In [24]:
users

,gender,age,occupation,cp
user_id,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455
...,...,...,...,...
6036,F,25,15,32603
6037,F,45,1,76006
6038,F,56,1,14706


In [25]:
occupations = occupations.set_index('occupation_id', drop=True)

occupations

,description
occupation_id,
0,other or not specified
1,academic/educator
2,artist
3,clerical/admin
4,college/grad student
5,customer service
6,doctor/health care
7,executive/managerial
8,farmer


Ahora, realizamos el merge de la siguiente manera:

In [26]:
pd.merge(users, occupations, left_on='occupation', right_index=True).sort_index()

,gender,age,occupation,cp,description
user_id,,,,,
1,F,1,10,48067,K-12 student
2,M,56,16,70072,self-employed
3,M,25,15,55117,scientist
4,M,45,7,02460,executive/managerial
5,M,25,20,55455,writer
...,...,...,...,...,...
6036,F,25,15,32603,scientist
6037,F,45,1,76006,academic/educator
6038,F,56,1,14706,academic/educator


Lo que hicimos entonces fue lo siguiente:

1. Usamos el método pd.merge.
1. Le pasamos el DataFrame users como "left".
1. Le pasamos el DataFrame occupations como "right".
1. Le indicamos que la tabla "left" va a usar la columna occupation para hacer el merge (left_on='occupation').
1. Le indicamos que la tabla "right" va a usar el índice para hacer el merge (right_index=True).
1. Ordenamos el resultado por índice de manera ascendente.

pd.merge hace por default un 'left join' que básicamente significa que la tabla "right" es la que se agrega a la tabla "left" y no al revés. Vamos a guardar nuestro resultado y a renombrar algunas columnas para que tengan más sentido:

In [27]:
users_full = pd.merge(users, occupations, left_on='occupation', right_index=True).sort_index()

In [28]:
users_full = users_full.rename(columns={'description': 'occupation', 'occupation': 'occupation_id'})

In [29]:
users_full.head()

,gender,age,occupation_id,cp,occupation
user_id,,,,,
1,F,1,10,48067,K-12 student
2,M,56,16,70072,self-employed
3,M,25,15,55117,scientist
4,M,45,7,02460,executive/managerial
5,M,25,20,55455,writer


Ahora, sabemos por el Readme.md que la columna age no indica en realidad la edad del usuario sino que es un código que indica en realidad un rango de edades. Los rangos de edades están en la tabla age_ranges, así que vamos a agregarlos:

In [30]:
age_ranges

,age_id,range
0,1,Under 18
1,18,18-24
2,25,25-34
3,35,35-44
4,45,45-49
5,50,50-55
6,56,56+


In [31]:
age_ranges = age_ranges.set_index('age_id', drop=True)
age_ranges

,range
age_id,
1,Under 18
18,18-24
25,25-34
35,35-44
45,45-49
50,50-55
56,56+


In [32]:
users_full = pd.merge(users_full, age_ranges, left_on='age', right_index=True).sort_index()

users_full.head()

,gender,age,occupation_id,cp,occupation,range
user_id,,,,,,
1,F,1,10,48067,K-12 student,Under 18
2,M,56,16,70072,self-employed,56+
3,M,25,15,55117,scientist,25-34
4,M,45,7,02460,executive/managerial,45-49
5,M,25,20,55455,writer,25-34


In [33]:
users_full = users_full.rename(columns={'age': 'age_id', 'range': 'age_range'})

users_full.head()

,gender,age_id,occupation_id,cp,occupation,age_range
user_id,,,,,,
1,F,1,10,48067,K-12 student,Under 18
2,M,56,16,70072,self-employed,56+
3,M,25,15,55117,scientist,25-34
4,M,45,7,02460,executive/managerial,45-49
5,M,25,20,55455,writer,25-34


Interesante, ¿no es así?

En este dataset tenemos varias categorías. Todavía ni siquiera hemos revisado las tablas ratings y movies (ésas se quedarán pendientes para los Retos), pero tan sólo en nuestro DataFrame users tenemos 3 categorías:

1. gender
1. age_range
1. occupation

Ahora que ya hemos aprendido a unir DataFrames, vamos a aprender una de las técnicas que tenemos para segmentar DataFrames por categorías. Veremos cómo un dataset tiene en realidad múltiples perspectivas y "dice" diferentes cosas dependiendo de cómo esté estructurado.

GroupBy

Hay múltiples formas de segmentar y dividir DataFrames. Una de ellas ya la hemos usado varias veces: los filtros. Existe otra manera que implica agrupar nuestros DataFrames por categorías usando el método groupby.

Por ejemplo, digamos que queremos segmentar nuestro DataFrame users por la columna gender. El primer paso sería hacer esto:

In [35]:
users_full['age_id'].value_counts()

age_id
25    2096
35    1193
18    1103
45     550
50     496
56     380
1      222
Name: count, dtype: int64

In [37]:
users_full.groupby('gender')

Ok... ¿y eso en qué nos ayuda? Ese objeto tiene la información necesaria para segmentar nuestro dataset usando la columna gender.

Ahora, tenemos que elegir qué columna (o columnas) queremos de regreso en cada grupo:

In [38]:
users_full.groupby('gender')['occupation']

Y para finalizar, tenemos que aplicar una función agregadora. ¿Recuerdas las agregaciones y reducciones que vimos anteriormente? Bueno, pues ésas son las funciones que podemos utilizar para agregar los datos de cada uno de nuestros grupos. Por ejemplo:

In [39]:
users_full.groupby('gender')['occupation'].size()

gender
F    1709
M    4331
Name: occupation, dtype: int64

size nos dice el tamaño de nuestros grupos. En este caso la columna occupation fue irrelevante, pero logramos obtener un conteo de cuántos hombres y cuántas mujeres hay en nuestro dataset.

Ahora podríamos hacer un conteo de cuántas mujeres y cuántos hombres tienen una cierta ocupación en nuestro dataset. value_counts toma cada una de las categorías en la Serie (en este caso occupation) y cuenta cuántas veces aparece cada categoría en cada grupo:



In [45]:
users_full.groupby('gender').count()

,age_id,occupation_id,cp,occupation,age_range
gender,,,,,
F,1709,1709,1709,1709,1709
M,4331,4331,4331,4331,4331


In [43]:
users_full.groupby('gender')['occupation'].value_counts()

gender  occupation            
F       college/grad student      234
        other or not specified    232
        academic/educator         209
        executive/managerial      139
        doctor/health care        102
        clerical/admin            100
        artist                     91
        homemaker                  89
        sales/marketing            79
        writer                     78
        K-12 student               66
        technician/engineer        52
        self-employed              51
        programmer                 50
        retired                    34
        customer service           31
        scientist                  28
        lawyer                     22
        unemployed                 15
        tradesman/craftsman         4
        farmer                      3
M       executive/managerial      540
        college/grad student      525
        other or not specified    479
        technician/engineer       450
        programmer 

¡Wow!

También podemos agrupar usando múltiples columnas. Por ejemplo, queremos hacer una primera agrupación por gender y luego, dentro de cada uno de los grupos, queremos volverlos a agrupar usando age_range. Después queremos saber cuántas entradas hay por subgrupo:

In [46]:
gender_age_range = users_full.groupby(['gender', 'age_range']).size()

In [47]:
gender_age_range

gender  age_range
F       18-24         298
        25-34         558
        35-44         338
        45-49         189
        50-55         146
        56+           102
        Under 18       78
M       18-24         805
        25-34        1538
        35-44         855
        45-49         361
        50-55         350
        56+           278
        Under 18      144
dtype: int64

Como puedes ver, obtenemos una Serie con un multiíndice. Esto quiere decir que para obtener un dato específico tenemos que indexarlo de esta manera:

In [48]:
gender_age_range.loc[('M', '45-49')]

361

Sabemos entonces que tenemos 361 hombres ("Male") en el rango de edad de 45 a 49.

Mucho más importante para el tema que nos concierne en este momento. Podríamos dividir nuestro dataset entre un dataset de hombres y otro de mujeres donde tenemos un conteo del número de personas que practican cada una de las ocupaciones:

In [49]:
gender_occupation_count = users_full.groupby(['gender'])['occupation'].value_counts()

In [50]:
male_occupation_count = gender_occupation_count.loc['M']
female_occupation_count = gender_occupation_count.loc['F']

In [51]:
male_occupation_count

occupation
executive/managerial      540
college/grad student      525
other or not specified    479
technician/engineer       450
programmer                338
academic/educator         319
sales/marketing           223
writer                    203
self-employed             190
artist                    176
doctor/health care        134
K-12 student              129
scientist                 116
retired                   108
lawyer                    107
customer service           81
clerical/admin             73
tradesman/craftsman        66
unemployed                 57
farmer                     14
homemaker                   3
Name: count, dtype: int64

Como ves, hemos creado un dataset distinto a partir de nuestro dataset original. Este nuevo dataset podría ser utilizado para visualizar, por ejemplo, cuántas personas tienen cada ocupación por género (eso lo haremos en alguno de nuestros Retos).

Ahora, no todas las funciones pueden ser utilizadas como agregaciones directamente, ya que no todas están implementadas en los objetos groupby. Pero podemos aumentar las posibilidades utilizando el método agg.

agg

Podemos nosotros utilizar algunas funciones que no están implementadas directamente, pasándole al método agg una función que tome una Serie o DataFrame como input y regrese una agregación. Por ejemplo, la función pd.Series.mode que nos regresa la categoría que más veces aparece dentro de nuestro grupo:

In [52]:
users_full.groupby(['gender'])['occupation'].agg(pd.Series.mode)

gender
F    college/grad student
M    executive/managerial
Name: occupation, dtype: object

In [53]:
users_full.groupby(['gender'])[['age_id', 'occupation_id']].agg(pd.Series.mode)

,age_id,occupation_id
gender,,
F,25,4
M,25,7


In [54]:
users_full.groupby(['gender'])[['age_id', 'occupation']].agg(pd.Series.mode)

,age_id,occupation
gender,,
F,25,college/grad student
M,25,executive/managerial


In [55]:
users_full.groupby(['gender'])[['age_range', 'occupation']].agg(pd.Series.mode)


,age_range,occupation
gender,,
F,25-34,college/grad student
M,25-34,executive/managerial


También podemos usar agg para aplicar varias funciones a nuestra Serie o DataFrame. Por ejemplo, vamos a computar el promedio, la desviación estándar y la mediana de las edades de nuestros usuarios (Por supuesto, en este caso estos cálculos no son correctos, ya que estamos utilizando códigos de edad que se refieren a rangos no a edades específicas):

In [56]:
users_full.columns

Index(['gender', 'age_id', 'occupation_id', 'cp', 'occupation', 'age_range'], dtype='object')

In [57]:
users_full = users_full[['gender', 'age_id', 'age_range', 'occupation_id', 'occupation', 'cp' ]]

users_full

,gender,age_id,age_range,occupation_id,occupation,cp
user_id,,,,,,
1,F,1,Under 18,10,K-12 student,48067
2,M,56,56+,16,self-employed,70072
3,M,25,25-34,15,scientist,55117
4,M,45,45-49,7,executive/managerial,02460
5,M,25,25-34,20,writer,55455
...,...,...,...,...,...,...
6036,F,25,25-34,15,scientist,32603
6037,F,45,45-49,1,academic/educator,76006
6038,F,56,56+,1,academic/educator,14706


In [59]:
users_full.groupby(['gender'])['age_id'].agg(['mean','std','median'])

,mean,std,median
gender,,,
F,30.859567,13.242564,25.0
M,30.552297,12.757110,25.0


Como ves, le pase a agg los nombres de las funciones como strings y aún así entendió qué es lo que quería.

También puedo usar agg para aplicarle funciones agregadoras a más de una columna en mi tabla:

In [60]:
users_full.groupby(['gender'])[['age_id','occupation_id']].agg(pd.Series.mode)

,age_id,occupation_id
gender,,
F,25,4
M,25,7


---

Lo importante de todo esto es la posibilidad que tenemos de segmentar nuestro dataset y generar información nueva usando agregaciones. Normalmente lo que haríamos con esta información nueva sería visualizarla de algún modo, y ya verás algunos ejemplos de eso durante la clase.

En esta sesión vimos poca información nueva. Esto se debe a que durante la clase haremos algunas prácticas un poco más extensas cuyo objetivo será aprovechar muchas de las herramientas que hemos aprendido durante todo el módulo.

¡Felicidades por haber llegado hasta aquí! La constancia y la disciplina son el único camino posible para convertirte en un gran Científico de Datos. Esperamos verte en el próximo módulo, donde tomaremos algunos datasets que ya han sido procesados (limpiados, transformados y estructurados) y los convertiremos en información estadística valiosa y hermosas visualizaciones.
